# 第21章 异步编程

## 21.1 核心概念

- 原生协程：使用`async def`定义的协程函数。在原生协程内可以使用`await`关键字委托另一个原生协程。`await`关键字不能再原生协程外部使用。
- 经典协程：一种生成器函数，在表达式中使用`yield`读取`my_coro.send(data)`调用发送的数据，可以使用`yield from`委托其他经典协程。
- 基于生成器的协程：使用`@type.coroutine`装饰的生成器函数。
- 异步生成器：使用`async def`定义，而且在主体中使用`yield`的生成器函数，返回一个提供`__anext__`方法的异步生成器对象。

## 21.2 一个asyncio示例：探测域名

In [1]:
import asyncio
import socket
from keyword import kwlist

MAX_KEYWORD_LEN = 4


async def probe(domain: str) -> tuple[str, bool]:
    # 返回一个元组，包含域名和一个布尔值，True表示域名可解析
    loop = asyncio.get_running_loop()
    try:
        # 获取asyncio事件循环的引用
        await loop.getaddrinfo(domain, None)
    except socket.gaierror:
        return (domain, False)
    return (domain, True)

async def main() -> None:
    names = (kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN)
    domains = (f'{name}.dev'.lower() for name in names)
    coros = [probe(domain) for domain in domains]
    for coro in asyncio.as_completed(coros):
        domain, found = await coro
        mark = '+' if found else ' '
        print(f'{mark} {domain}')

In [2]:
await main()

+ del.dev
+ try.dev
+ and.dev
+ as.dev
+ in.dev
+ def.dev
+ not.dev
  none.dev
+ from.dev
  true.dev
  else.dev
  if.dev
  is.dev
  with.dev
  pass.dev
  elif.dev
  or.dev
  for.dev


## 21.3 使用asyncio和HTTPX下载

In [3]:
import time
from pathlib import Path
from typing import Callable

import httpx

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

# 存放国旗图像的网络资源
BASE_URL = 'https://www.fluentpython.com/data/flags'
# 保存图像的本地目录
DEST_DIR = Path('downloaded')                         

def save_flag(img: bytes, filename: str) -> None: 
    (DEST_DIR / filename).write_bytes(img)

def main(downloader: Callable[[list[str]], int]) -> None:
    # 创建目录
    DEST_DIR.mkdir(exist_ok=True)                          
    # 记录并报告downloader运行时间
    t0 = time.perf_counter()                               
    count = downloader(POP20_CC)
    elapsed = time.perf_counter() - t0
    print(f'\n{count} downloads in {elapsed:.2f}s')

In [9]:
import asyncio

from httpx import AsyncClient

def download_many(cc_list: list[str]) -> int:
    return asyncio.run(supervisor(cc_list))    

async def supervisor(cc_list: list[str]) -> int:
    # AsyncClient提供了异步设置和清理方法的上下文管理器
    async with AsyncClient() as client:          
        to_do = [download_one(client, cc)
                 for cc in sorted(cc_list)]      
        # 等待全部执行完毕，以可异步调用对象的提交顺序返回结果列表
        res = await asyncio.gather(*to_do)       

    return len(res)                              

In [10]:
async def download_one(client: AsyncClient, cc: str):
    image = await get_flag(client, cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc

async def get_flag(client: AsyncClient, cc: str) -> bytes:
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    # httpx.AsyncClient实例的get方法返回一个ClientResponse对象
    resp = await client.get(url, timeout=6.1, follow_redirects=True)  
    return resp.read()

## 21.4 增强asyncio版下载脚本的功能

功能：把几个协程传给`asyncio.gather`，按照协程的提交顺序返回协程的结果构成的列表。

代码：`assets/20-executors/getflags/flags2_asyncio.py`

Python中的信号量：
- `asyncio.Semaphore`有一个内部计数器，每次使用`await`处理协程方法`.acquire()`，计数器递减；每次调用`.release()`方法，计数器递增。
- 若计数器大于零，则使用`await`处理`.acquire()`方法没有延迟；若计数器为零，则`.acquire()`中止待处理的协程，直到其他协程在同一个`Semaphore`实例上调用`.release()`，递增计数器。

## 21.5 使用asyncio编写服务器

功能：用户通过unicodedata模块中Unicode字符标准名称所包含的单词，在服务器中查询Unicode字符，游览器显示搜索结果。

项目启动：`nvicorn web_mojifinder:app --reload`
- `web_mojifinder:app`：包名、一个冒号和包内定义的ASGI应用程序的名称。
- `--reload`：让uvicorn监控应用程序源文件的变化，自动重新加载。只在开发过程中有用。

## 21.6 异步迭代和异步可迭代对象

- `async for`处理异步可迭代对象，实现了`__aiter__`对象，然而`__aiter__`必须是常规方法（不是协程方法），而且必须返回一个异步迭代器。
- 异步迭代器提供`__anext__`协程方法，返回一个可异步调用对象，也实现`__aiter__`，返回`self`。

### 21.6.1 异步生成器函数

1. Python异步控制台，使用命令行选项`-m asyncio`运行解释器，可以得到一个异步REPL。

2. 实现一个异步生成器：使用`async for`实现。

In [17]:
import asyncio
import socket
from collections.abc import Iterable, AsyncIterator
from typing import NamedTuple, Optional


class Result(NamedTuple): 
    domain: str
    found: bool


# 设置别名 
OptionalLoop = Optional[asyncio.AbstractEventLoop]


async def probe(domain: str, loop: OptionalLoop = None) -> Result:
    if loop is None:
        loop = asyncio.get_running_loop()
    try:
        await loop.getaddrinfo(domain, None)
    except socket.gaierror:
        return Result(domain, False)
    return Result(domain, True)

# 异步生成器函数产生一个异步生成器对象
async def multi_probe(domains: Iterable[str]) -> AsyncIterator[Result]:  
    loop = asyncio.get_running_loop()
    # 构建一个probe协程对象列表，对应各个域名
    coros = [probe(domain, loop) for domain in domains]
    for coro in asyncio.as_completed(coros):
        # 等待协程对象，获取结果
        result = await coro 
        yield result  

In [18]:
async def main(tld: str) -> None:
    tld = tld.strip('.')
    # 生成长度不超过4的关键字
    names = (kw for kw in kwlist if len(kw) <= 4)
    # 使用指定的顶级域名生成一组域名
    domains = (f'{name}.{tld}'.lower() for name in names)
    print('FOUND\t\tNOT FOUND')  
    print('=====\t\t=========')
    async for domain, found in multi_probe(domains):  
        indent = '' if found else '\t\t'
        print(f'{indent}{domain}')

In [16]:
await main('org')

FOUND		NOT FOUND
=====		=========
pass.org
def.org
in.org
for.org
not.org
true.org
del.org
none.org
from.org
or.org
and.org
elif.org
try.org
else.org
with.org
		is.org
		as.org
		if.org


3. 异步生成器用作上下文管理器。

In [22]:
from contextlib import asynccontextmanager

def download_webpage(url):
    print('Download webpage', url)

def update_stats(url):
    print('Update stats', url)
    
def process(data):
    print('Process', data)
    

@asynccontextmanager
async def web_page(url):
    loop = asyncio.get_running_loop()
    data = await loop.run_in_executor(None, download_webpage, url)
    yield data
    await loop.run_in_executor(None, update_stats, url)
    
async with web_page('google.com') as data:
    process(data)

Download webpage google.com
Process None
Update stats google.com


4. 异步生成器和原生协程之间的主要异同点
   - 都使用`async def`声明。
   - 异步生成器的主体中肯定有一个`yield`表达式，原生协程中不会有。
   - 原生协程可能返回`None`之外的值，异步生成器只能使用空`return`语句。
   - 原生协程是可异步调用对象，可由`await`表达式驱动，也可以传给`asyncio`库中众多接受可异步调用对象的函数。异步生成器不是可异步调用对象，而是异步可迭代对象，由`async for`或异步推导式驱动。

### 21.6.2 异步生成器表达式和异步推导式

- 异步生成器表达式可在程序的任何位置定义，但是只能在原生协程或异步生成器函数内使用。
- 异步推导式只能出现在`async def`主体内，或者是异步控制台中。

## 21.7 asyncio之外的异步世界：Curio

使用Curio重写blogdom.py脚本

In [23]:
from curio import run, TaskGroup
import curio.socket as socket
from keyword import kwlist

MAX_KEYWORD_LEN = 4


async def probe(domain: str) -> tuple[str, bool]: 
    try:
        await socket.getaddrinfo(domain, None)
    except socket.gaierror:
        return (domain, False)
    return (domain, True)

async def main() -> None:
    names = (kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN)
    domains = (f'{name}.dev'.lower() for name in names)
    # TaskGroup用于监控和控制多个协程，确保协程全部执行并得到清理
    async with TaskGroup() as group:
        for domain in domains:
            # 启动协程，由TaskGroup实例管理
            await group.spawn(probe, domain)
        # 迭代TaskGroup，完成任务之后产出Task实例    
        async for task in group:
            domain, found = task.result
            mark = '+' if found else ' '
            print(f'{mark} {domain}')

In [24]:
run(main())

+ as.dev
+ and.dev
+ def.dev
+ in.dev
+ del.dev
+ not.dev
+ from.dev
+ try.dev
  true.dev
  elif.dev
  else.dev
  if.dev
  is.dev
  none.dev
  with.dev
  pass.dev
  or.dev
  for.dev


Curio的特色：
- Curio中的`TaskGroup`是异步上下文管理器，取代了`asyncio`的多个特有API和编程模式。任务组支持结构化并发。
- 对在同一份基准代码中使用协程和线程编程提供了更好的支持。
- 提供了一个`UniversalQueue`，可用于协调线程、Curio协程和`asyncio`协程之间的工作。

## 21.8 异步原理与陷阱

陷阱：
- 规范的异步编程方法可以提升服务器的性能。
- 没有所谓的“I/O密集型系统”，可能遇到的是I/O密集型函数。

发现CPU占用出现瓶颈后，可以采取以下措施：
- 把任务委托给Python进程池。
- 把任务委托给外部任务队列。
- 使用Cython、C、Rust，或者可编译成机器码、能与Python/C API交互的其他语言重写相关代码。
- 如果判断自己可以承受性能损失，可以什么都不做，把决定记录下来，方便以后重新审视。

## 21.9 杂谈

uvloop的基准测试：
- 2016年，Yury Selivanov发布了uvloop，需要配置Go语言使用单个线程，使用httptools。
- 使用aiohttp对uvloop进行了基准测试之后，编写httptools。理由是HTTP解析器的速度太慢了。
- HTTP性能测试用到了一个简单的回显服务器，HTTP首部解析是CPU密集型操作，使用Python编写的函数解析首部，导致事件循环受到阻塞。因此优化httptools库。

系统性能诊断：
- 背景：系统使用Python2.7编写，使用Twisted框架，属于asyncio的前身。
- 问题：每增加一个功能，有更多CPU密集型代码拖慢Twisted的事件循环。
- 解决思路：重新思考系统架构，重写大量代码，也行需要任务队列，可能还要使用微服务，或者以更适合并发处理CPU密集型操作的语言编写库。
- 结论：项目方不准备继续投资了，项目被取消了。